# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url =  f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2427039872549,
 'week52high': 154.55,
 'week52low': 102.7,
 'week52highSplitAdjustOnly': 154,
 'week52lowSplitAdjustOnly': 108,
 'week52change': 0.33510158583237226,
 'sharesOutstanding': 17154944091,
 'float': 0,
 'avg10Volume': 59534794,
 'avg30Volume': 85110473,
 'day200MovingAvg': 137.21,
 'day50MovingAvg': 149.1,
 'employees': 153031,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.866696998989794,
 'dividendYield': 0.006049155493760174,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-24',
 'nextEarningsDate': '2021-10-25',
 'peRatio': 28.70523252284206,
 'beta': 1.550545107904781,
 'maxChangePercent': 57.39448202971594,
 'year5ChangePercent': 4.861034918993753,
 'year2ChangePercent': 2.061684190351497,
 'year1ChangePercent': 0.2951355923760491,
 'ytdChangePercent': 0.10970198308038266,
 'month6ChangePercent': 0.0747092470141581,
 'month3ChangePercent': 0.13623309490199648,
 'month1ChangePercent': 0.030967830346547168,
 'day30ChangePercent': 0

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.2951355923760491

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe           
            
            


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.30,0.624948,N/A
1,AAL,21.87,0.627493,N/A
2,AAP,208.09,0.36947,N/A
3,AAPL,146.41,0.306405,N/A
4,ABBV,118.47,0.298435,N/A
...,...,...,...,...
500,YUM,137.94,0.51482,N/A
501,ZBH,153.00,0.127907,N/A
502,ZBRA,562.70,1.000589,N/A
503,ZION,56.65,0.776104,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [14]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

/home/bhois/anaconda3/envs/algo-trading/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.86,2.352105,N/A
1,441,TPR,44.34,2.237073,N/A
2,23,ALB,226.88,1.798555,N/A
3,106,COF,166.31,1.6923,N/A
4,129,DFS,132.24,1.675906,N/A
5,272,KSS,53.48,1.635554,N/A
6,410,SIVB,592.00,1.575688,N/A
7,179,FCX,38.53,1.545676,N/A
8,344,NUE,106.31,1.534526,N/A
9,453,UAA,26.00,1.520665,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [21]:
def portfolio_input() -> float:
    try:
        portfolio_size = float(input('Enter the size of your portfolio:'))
    except ValueError:
        print('That is not a number!')
        return None
    
    return portfolio_size

portfolio_size = portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000.0


In [25]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

/home/bhois/anaconda3/envs/algo-trading/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,82.86,2.352105,241
1,441,TPR,44.34,2.237073,451
2,23,ALB,226.88,1.798555,88
3,106,COF,166.31,1.6923,120
4,129,DFS,132.24,1.675906,151
5,272,KSS,53.48,1.635554,373
6,410,SIVB,592.00,1.575688,33
7,179,FCX,38.53,1.545676,519
8,344,NUE,106.31,1.534526,188
9,453,UAA,26.00,1.520665,769


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [60]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,                    
                    data[symbol]['price'],                                        
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'] if data[symbol]['stats']['year1ChangePercent'] != None else 0.0,
                    'N/A',                    
                    data[symbol]['stats']['month6ChangePercent'] if data[symbol]['stats']['month6ChangePercent'] != None else 0.0,
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'] if data[symbol]['stats']['month3ChangePercent'] != None else 0.0,                    
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'] if data[symbol]['stats']['month1ChangePercent'] != None else 0.0,
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )

pd.set_option('display.max_rows', None)
hqm_dataframe        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,161.100,N/A,0.618720,N/A,0.276583,N/A,0.167924,N/A,0.043321,N/A,N/A
1,AAL,21.590,N/A,0.634991,N/A,0.233311,N/A,-0.045899,N/A,0.035334,N/A,N/A
2,AAP,211.170,N/A,0.368210,N/A,0.320855,N/A,0.011619,N/A,-0.018217,N/A,N/A
3,AAPL,147.600,N/A,0.325336,N/A,0.073672,N/A,0.130213,N/A,0.012898,N/A,N/A
4,ABBV,119.980,N/A,0.306966,N/A,0.080093,N/A,-0.000217,N/A,-0.009166,N/A,N/A
5,ABC,124.780,N/A,0.205247,N/A,0.113673,N/A,0.001899,N/A,0.050101,N/A,N/A
6,ABMD,363.020,N/A,0.113194,N/A,0.056619,N/A,0.167385,N/A,0.035046,N/A,N/A
7,ABT,125.860,N/A,0.232793,N/A,-0.007759,N/A,0.031201,N/A,0.023274,N/A,N/A
8,ACN,327.480,N/A,0.409576,N/A,0.279948,N/A,0.111252,N/A,0.044443,N/A,N/A
9,ADBE,651.060,N/A,0.421941,N/A,0.294969,N/A,0.297744,N/A,0.043821,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [90]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:    
    for time_period in time_periods:        
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(
                hqm_dataframe[f'{time_period} Price Return'],
                hqm_dataframe.loc[row, f'{time_period} Price Return'])/100        
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTNT,318.69,62,1.336988,0.9,0.998381,0.98,0.450931,1.0,0.200109,0.98,99.059406
1,ALB,230.98,86,1.760419,0.98,0.457467,0.78,0.402296,0.96,0.340904,1.0,98.960396
2,IT,301.47,66,1.285629,0.86,0.800020,0.94,0.256753,0.72,0.144617,0.8,97.079208
3,CARR,56.96,351,0.973756,0.68,0.456583,0.76,0.261725,0.8,0.155455,0.88,95.792079
4,LB,80.22,249,2.318185,1.0,0.827654,0.96,0.218248,0.66,0.081943,0.44,95.49505
5,HCA,253.08,79,0.973360,0.66,0.438295,0.66,0.196706,0.56,0.155543,0.9,94.50495
6,MSCI,647.40,30,0.749525,0.38,0.491146,0.84,0.307484,0.86,0.137774,0.76,93.811881
7,CBRE,100.79,198,1.245279,0.82,0.453436,0.74,0.126198,0.34,0.153409,0.86,93.168317
8,IPG,37.33,535,1.095683,0.8,0.449742,0.72,0.114093,0.3,0.147858,0.82,92.227723
9,WAT,400.30,49,0.868997,0.58,0.401789,0.5,0.257785,0.74,0.085207,0.48,91.732673


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [62]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean (momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,161.100,N/A,0.618720,75.841584,0.276583,76.831683,0.167924,90.09901,0.043321,66.732673,77.376238
1,AAL,21.590,N/A,0.634991,77.227723,0.233311,69.108911,-0.045899,34.455446,0.035334,62.970297,60.940594
2,AAP,211.170,N/A,0.368210,46.930693,0.320855,83.960396,0.011619,53.267327,-0.018217,28.118812,53.069307
3,AAPL,147.600,N/A,0.325336,40.990099,0.073672,27.326733,0.130213,82.772277,0.012898,46.930693,49.50495
4,ABBV,119.980,N/A,0.306966,38.613861,0.080093,28.712871,-0.000217,47.524752,-0.009166,32.277228,36.782178
5,ABC,124.780,N/A,0.205247,27.128713,0.113673,38.613861,0.001899,50.09901,0.050101,72.871287,47.178218
6,ABMD,363.020,N/A,0.113194,17.623762,0.056619,23.366337,0.167385,89.70297,0.035046,62.574257,48.316832
7,ABT,125.860,N/A,0.232793,30.693069,-0.007759,11.089109,0.031201,59.207921,0.023274,54.455446,38.861386
8,ACN,327.480,N/A,0.409576,53.861386,0.279948,77.425743,0.111252,79.80198,0.044443,68.514851,69.90099
9,ADBE,651.060,N/A,0.421941,56.039604,0.294969,80.594059,0.297744,97.227723,0.043821,67.524752,75.346535


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [74]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTNT,318.69,62,1.336988,97.425743,0.998381,99.80198,0.450931,99.80198,0.200109,99.207921,99.059406
1,ALB,230.98,86,1.760419,99.60396,0.457467,97.227723,0.402296,99.009901,0.340904,100.0,98.960396
2,IT,301.47,66,1.285629,96.633663,0.800020,99.405941,0.256753,95.445545,0.144617,96.831683,97.079208
3,CARR,56.96,351,0.973756,92.079208,0.456583,97.029703,0.261725,96.435644,0.155455,97.623762,95.792079
4,LB,80.22,249,2.318185,100.0,0.827654,99.60396,0.218248,93.861386,0.081943,88.514851,95.49505
5,HCA,253.08,79,0.973360,91.881188,0.438295,96.039604,0.196706,92.277228,0.155543,97.821782,94.50495
6,MSCI,647.40,30,0.749525,83.762376,0.491146,97.821782,0.307484,97.425743,0.137774,96.237624,93.811881
7,CBRE,100.79,198,1.245279,96.237624,0.453436,96.831683,0.126198,82.178218,0.153409,97.425743,93.168317
8,IPG,37.33,535,1.095683,94.653465,0.449742,96.633663,0.114093,80.594059,0.147858,97.029703,92.227723
9,WAT,400.30,49,0.868997,88.712871,0.401789,93.267327,0.257785,95.643564,0.085207,89.306931,91.732673


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [67]:
portfolio_size = portfolio_input()

Enter the size of your portfolio:1000000


In [75]:
position_size = portfolio_size/len(hqm_dataframe.index)
position_size

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTNT,318.69,62,1.336988,97.425743,0.998381,99.80198,0.450931,99.80198,0.200109,99.207921,99.059406
1,ALB,230.98,86,1.760419,99.60396,0.457467,97.227723,0.402296,99.009901,0.340904,100.0,98.960396
2,IT,301.47,66,1.285629,96.633663,0.800020,99.405941,0.256753,95.445545,0.144617,96.831683,97.079208
3,CARR,56.96,351,0.973756,92.079208,0.456583,97.029703,0.261725,96.435644,0.155455,97.623762,95.792079
4,LB,80.22,249,2.318185,100.0,0.827654,99.60396,0.218248,93.861386,0.081943,88.514851,95.49505
5,HCA,253.08,79,0.973360,91.881188,0.438295,96.039604,0.196706,92.277228,0.155543,97.821782,94.50495
6,MSCI,647.40,30,0.749525,83.762376,0.491146,97.821782,0.307484,97.425743,0.137774,96.237624,93.811881
7,CBRE,100.79,198,1.245279,96.237624,0.453436,96.831683,0.126198,82.178218,0.153409,97.425743,93.168317
8,IPG,37.33,535,1.095683,94.653465,0.449742,96.633663,0.114093,80.594059,0.147858,97.029703,92.227723
9,WAT,400.30,49,0.868997,88.712871,0.401789,93.267327,0.257785,95.643564,0.085207,89.306931,91.732673


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [81]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [82]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():    
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [89]:
writer.save()